In [40]:
from models.create_dataset import create_dataset
import pandas as pd
import numpy as np
import logging
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
# import tensor dataset
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
# import MAE
import pickle
from sklearn.metrics import mean_absolute_error

In [41]:
# Set up logging
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
                    , handlers=[logging.FileHandler('LSTM_epf.log'), logging.StreamHandler()])
logger = logging.getLogger(__name__)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [43]:
# load data
target_col = 'DK1_price'
df = create_dataset(target_col=target_col)

['NO_Solar', 'NO_Solar_lag_1', 'NO_Solar_lag_2', 'NO_Solar_lag_3', 'NO_Solar_lag_7']


In [44]:
# split into X and y
y = df[target_col]
X = df.drop(target_col, axis=1)

In [45]:
# Pivot hourly index out to columns so index is only date
pivot_columns = [col for col in X.columns if not col.startswith('day_of_week')]
X = X.pivot_table(index=X.index.date, columns=X.index.hour, values=pivot_columns)
X = X.dropna()
# Some hours will only have 0 values, drop these columns (e.g. Solar)
X = X.loc[:, (X != 0).any(axis=0)]
# and some are 0 almost always, drop features with a MAD below threshold
X = X.loc[:, X.sub(X.median(axis=0), axis=1).abs().median(axis=0) > 0.01]


In [46]:
# now y
# make y to dataframe first
y = y.rename('Price')
y = pd.DataFrame(y)
y = y.pivot_table(index=y.index.date, columns=y.index.hour, values=y.columns)
# join multiindex columns to one, price with hour number
y = y.dropna()
# rename to 'Price' for some reason for this toolbox to work

In [47]:
##### temporary start cut off as well to see if it works
#start_cutoff = pd.to_datetime('2019-01-01 00:00')
val_cutoff = pd.to_datetime('2020-07-01')
test_cutoff = pd.to_datetime('2021-01-01')
#X_train = X.loc[(X.index >= start_cutoff) & (X.index < val_cutoff)] ########## temporary
X_train = X.loc[X.index < val_cutoff]
X_val = X.loc[(X.index >= val_cutoff) & (X.index < test_cutoff)]
X_test = X.loc[X.index >= test_cutoff]
#y_train = y.loc[(y.index < val_cutoff) & (y.index >= start_cutoff)] ########### temporary
y_train = y.loc[y.index < val_cutoff]
y_val = y.loc[(y.index >= val_cutoff) & (y.index < test_cutoff)]
y_test = y.loc[y.index >= test_cutoff]

C:\Users\frede\AppData\Local\Temp\ipykernel_26468\1260850503.py:5: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  X_train = X.loc[(X.index >= start_cutoff) & (X.index < val_cutoff)] ########## temporary
C:\Users\frede\AppData\Local\Temp\ipykernel_26468\1260850503.py:6: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  X_train = X.loc[X.index < val_cutoff]
C:\Users\frede\AppData\Local\Temp\ipykernel_26468\1260850503.py:7: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparab

In [48]:
class Scaler():

    def __init__(self, median=None, mad=None):
        self.median = None
        self.mad = None

    def fit(self, data):
        if isinstance(data, pd.Series):
            data = pd.DataFrame(data)
        self.median = data.median(axis=0).to_numpy()
        # calculate median absolute deviation
        self.mad = data.sub(data.median(axis=0), axis=1).abs().median(axis=0).to_numpy()
        # print na in mad
        return self

    def transform(self, data):
        if self.median is None or self.mad is None:
            raise ValueError('Fit scaler first')

        if isinstance(data, pd.Series):
            data = pd.DataFrame(data)
        X_transformed = data.sub(self.median, axis=1)
        X_transformed = X_transformed.div(self.mad, axis=1)
        X_transformed = np.arcsinh(X_transformed)

        return X_transformed

    def inverse_transform(self, data):

        if self.median is None or self.mad is None:
            raise ValueError('Fit scaler first')
        # fix so this works for series and dataframe
        if isinstance(data, pd.Series):
            data = pd.DataFrame(data)

        X_inversed = np.sinh(data)
        X_inversed = X_inversed.mul(self.mad, axis=1)
        X_inversed = X_inversed.add(self.median, axis=1)
        # make this work for series


        return X_inversed

In [50]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_().to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).requires_grad_().to(device)
        out, (hn, cn) = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [51]:
# Create sequences
def create_sequences(X, y, sequence_length):
    X_sequences = []
    y_sequences = []
    for i in range(len(X) - sequence_length):
        X_sequences.append(X[i:i+sequence_length])
        y_sequences.append(y[i+sequence_length])
    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)
    X_sequences = torch.tensor(X_sequences).float()
    y_sequences = torch.tensor(y_sequences).float()
    return X_sequences, y_sequences

In [52]:
# Tree structured parzen estimator

optimize_hyperparameters = True

batch_size = [2, 4, 8, 16, 32, 64, 128, 256]
num_layers = [1, 2, 3, 4, 5]
seq_length = [1, 2, 3, 4, 6, 8, 12, 24]
if optimize_hyperparameters:
    from hyperopt import fmin, tpe, hp, Trials

    epochs = 50
    criterion = nn.MSELoss() # maybe not this one, but for now

    # Define the search space
    space = {
        'weight_decay': hp.loguniform('weight_decay', -10, -1),
        'num_layers': hp.choice('num_layers', num_layers),
        'sequence_length': hp.choice('sequence_length', seq_length), # 1-24 hours
        'batch_size': hp.choice('batch_size', batch_size),
        'learning_rate': hp.loguniform('learning_rate', -8, -1),
        'hidden_size': hp.quniform('hidden_size', 32, 512, 32),
        'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
    }

    # Define the objective function
    def objective(params, input_dim=X_train.shape[1], output_dim=24):
        # Train and evaluate your model with the given hyperparameters
        # Return the validation accuracy or other metric you want to optimize
        #print(f'num_layers: {params["num_layers"]}, batch_size:{params["batch_size"]}, seq_len{params["sequence_length"]}')
        #params['num_layers'] = num_layers[params['num_layers']]
        params['batch_size'] = int(params['batch_size'])
        params['hidden_size'] = int(params['hidden_size'])
        params['num_layers'] = int(params['num_layers'])
        params['sequence_length'] = int(params['sequence_length'])
        if params['num_layers'] == 1: # if only one layer, no dropout
            params['dropout_rate'] = 0

        #model = LSTM(input_dim, output_dim, , params['dropout_rate']).to(device)
        model = LSTM(input_size=input_dim, hidden_size=params['hidden_size'], num_layers=params['num_layers'], output_size=output_dim, dropout=params['dropout_rate']).to(device)
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
        # print layers
        # __loader__
        # fit scaler first, then transform: Fitting on training data then transforming on training and validation data

        XScaler = Scaler()
        # fit scaler without last 7 features
        XScaler.fit(X_train.iloc[:, :-7])


        X_train_scaled = XScaler.transform(X_train.iloc[:, :-7])
        # add dummies
        X_train_scaled = pd.concat([X_train_scaled, X_train.iloc[:, -7:]], axis=1)
        X_val_scaled = XScaler.transform(X_val.iloc[:, :-7])
        X_val_scaled = pd.concat([X_val_scaled, X_val.iloc[:, -7:]], axis=1)

        yScaler = Scaler()
        yScaler.fit(y_train)
        y_train_scaled = yScaler.transform(y_train)
        y_val_scaled = yScaler.transform(y_val)
        #train_dataset = Dataset()
        #train_dataset.load_data(X_train_scaled, y_train_scaled)
        #train_loader = DataLoader(dataset=list(zip(X_train_scaled.values, y_train_scaled.values)), batch_size=params['batch_size'], shuffle=False)
        # make torch dataset
        X_train_sequences, y_train_sequences = create_sequences(X_train_scaled.values, y_train_scaled.values, params['sequence_length'])
        X_val_sequences, y_val_sequences = create_sequences(X_val_scaled.values, y_val_scaled.values, params['sequence_length'])
        train_dataset = TensorDataset(X_train_sequences, y_train_sequences)
        val_dataset = TensorDataset(X_val_sequences, y_val_sequences)
        train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)




        train_losses = []
        val_losses = []
        best_val_loss = np.inf
        for epoch in range(epochs):
            train_loss = 0.0
            val_loss = 0.0
            model.train()
            for i, (inputs, labels) in enumerate(train_loader):
                # transfer to GPU
                inputs, labels = inputs.float().to(device), labels.float().to(device)


                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                # print statistics
                train_loss += loss.item()

            model.eval()
            for i, (inputs, labels) in enumerate(val_loader):
                # transfer to GPU

                inputs, labels = inputs.float().to(device), labels.float().to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

            train_losses.append(train_loss/len(train_loader))
            val_losses.append(val_loss/len(val_loader))
            # early stopping
            if epoch % 5 == 0:

                # if val hasn't decreased for 5 epochs, stop
                if min(val_losses[-3:]) > best_val_loss:
                    break
                best_val_loss = min(val_losses[-3:])

        #accuracy = val_losses[-1]
        # min of last 3 val losses
        accuracy = min(val_losses[-3:])

        return accuracy

    # Define the TPE algorithm
    tpe_algorithm = tpe.suggest

    # Define the number of iterations
    max_evals = 1000

    # Initialize the trials object
    trials = Trials()

    # Run the TPE algorithm to optimize the hyperparameters
    best_params = fmin(objective, space, algo=tpe_algorithm, max_evals=max_evals, trials=trials, verbose=True)

    # Print the best hyperparameters
    print("Best hyperparameters:", best_params)
    # save best parameters to json
    param_path = r'C:\Users\frede\PycharmProjects\Masters\models\LSTM\best_params.pkl'
    pickle.dump(best_params, open(param_path, 'wb'))
    trials_path = r'C:\Users\frede\PycharmProjects\Masters\models\LSTM\trials.pkl'
    pickle.dump(trials, open(trials_path, 'wb'))


100%|██████████| 1000/1000 [10:49:51<00:00, 38.99s/trial, best loss: 1.683862618181143]  
Best hyperparameters: {'batch_size': 0, 'dropout_rate': 0.07581341030688578, 'hidden_size': 320.0, 'learning_rate': 0.004557564896692678, 'num_layers': 4, 'sequence_length': 4, 'weight_decay': 0.004622173580033262}


In [53]:
2+2

4

In [55]:
best_params['batch_size'] = batch_size[best_params['batch_size']]

In [57]:
best_params['num_layers'] = num_layers[best_params['num_layers']]

In [59]:
best_params['sequence_length'] = seq_length[best_params['sequence_length']]

In [58]:
seq_length[best_params['sequence_length']]

6

In [61]:
print("Best hyperparameters:", best_params)
# save best parameters to json
param_path = r'C:\Users\frede\PycharmProjects\Masters\models\LSTM\best_params.pkl'
pickle.dump(best_params, open(param_path, 'wb'))
trials_path = r'C:\Users\frede\PycharmProjects\Masters\models\LSTM\trials.pkl'
pickle.dump(trials, open(trials_path, 'wb'))

Best hyperparameters: {'batch_size': 2, 'dropout_rate': 0.07581341030688578, 'hidden_size': 320.0, 'learning_rate': 0.004557564896692678, 'num_layers': 5, 'sequence_length': 6, 'weight_decay': 0.004622173580033262}


In [62]:
Trials.best_trial

In [63]:
trials.best_trial['result']['loss']

1.683862618181143